In [ ]:
from Parameters import ModelParams
from Mass_Loss_Class import MassLoss
from Fractionation_Class import Fractionation
from Model_Data_Loader import ModelDataLoader
from Misc_Class import Misc
from Star_Parameters import StarParams

import numpy as np
import pandas as pd
import json

In [ ]:
# ---------- Initialize parameters and classes ----------
params = ModelParams()
mass_loss = MassLoss(params)
fractionation = Fractionation(params)
star_params = StarParams(params)
misc = Misc(params)
loader = ModelDataLoader('/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/', params)

# ---------- Load data ----------
### option 1
mass, radius, Teq = loader.load_single_ddat_file('3HHe_subNeptune.ddat')

### option 2
# r_m_T = loader.load_all_ddat_files()
# Convert units from the r_m_T dictionary to cgs

### option 3
# radius = 1 # cm
# mass = 1 # g
# Teq = 1 # K

### option 4
# with open("planet_params.json", "r") as file:
#     planet_params = json.load(file)
# selected_planet = "TOI-431 b" # change the planet here

# if selected_planet in planet_params:
#     new_feuv = planet_params[selected_planet]["FEUV"]
#     params.update_param("FEUV", new_feuv)
#     print(f"Updated FEUV for {selected_planet}: {params.get_param('FEUV'):.2e} erg cm-2 s-1")
# else:
#     print(f"Make sure the planet you selected exists in the list! Check the spelling.")

# mearth = params.mearth
# rearth = params.rearth
# radius = np.array([planet_params[selected_planet]["radius"] * rearth]) # cm
# mass = np.array([planet_params[selected_planet]["mass"] * mearth]) # g
# Teq = np.array([planet_params[selected_planet]["teq"]]) # K

### find unique temperatures
unique_Teqs = np.unique(Teq)
print("Unique Teq values found:", unique_Teqs)

# ---------- Range of EUV flux values ----------
all_flux_results = []

for Teq_val in unique_Teqs:
    star_params.update_param("Teq", Teq_val)
    flux_range = star_params.get_FEUV_range_any_age()
    # flux_range = misc.get_flux_range(Teq_val)

    mask = np.isclose(Teq, Teq_val)
    mass_group = mass[mask]
    radius_group = radius[mask]
    teq_group = Teq[mask]
    print(f"\nProcessing group with Teq = {Teq_val} K")

    # ---------- Execute ----------
    for flux in flux_range:
        print(f"\n Running model with FEUV = {flux} erg/cm^2/s")
        params.update_param('FEUV', flux)

        # ---------- Mass loss model ----------
        mass_loss_results = mass_loss.compute_mass_loss_parameters(mass_group, radius_group, teq_group)
        # mdot = mass_loss_results[0]['Mdot']
        # print(f"Mdot =", mdot)

        results_for_this_flux = {'FEUV': flux, 'Teq': Teq_val, 'end_results': mass_loss_results}
        
        # ---------- Fractionation model ----------
        ### option 1 - without self consistent feedback loop
        # fractionation_results = fractionation.execute_fractionation(mass_loss_results, misc)
        
        ### option 2 - with feedback loop
        # fractionation_results = fractionation.execute_self_consistent_fractionation(mass_loss_results, mass_loss, misc, params)
        # results_for_this_flux = {'FEUV': flux, 'Teq': Teq_val, 'end_results': fractionation_results}
        
        all_flux_results.append(results_for_this_flux)

In [ ]:
# save data for plotting
rows = []

for flux_data in all_flux_results:
    flux = flux_data['FEUV']
    for res in flux_data['end_results']:
        record = {'FEUV': flux}
        record.update(res)
        rows.append(record)

df = pd.DataFrame(rows)
df.to_csv('/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/3HHe_subNeptune.csv', index=False)